# Obteniendo el TOTAL de vacunados y fallecidos por COVID-19 por cada semana epidemiológica de todo el Perú

En este notebook se busca obtener un dataset con el total de **fallecidos** y vacunados **(dosis 1,2 y 3)** por cada **año** y **semana epidemiológica** de todo el Perú. 

Se considera como **completamente vacunado** a todas las personas reciberon 2 dosis de vacunación.

## 0. Cargar librerías

In [59]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from epiweeks import Week

import functions as fn

## 1. Cargamos direcciones de RawData

El dataset de vacunados **'TB_VACUNACION_COVID19.csv'** no está incluido en el repositorio de GitHub debido a su excesivo tamaño (~2gb). Sin embargo se puede descargar directamente desde la página de Datos Abiertos de COVID-19 de MINSA, [Vacunación contra la COVID-19](https://www.datosabiertos.gob.pe/dataset/vacunacion) y colocarlo en la dirección RawData.

In [60]:
vac_url = 'RawData/TB_VACUNACION_COVID19.csv'
fal_url = 'RawData/fallecidos_covid.csv'

## 2. Procesar el dataset de fallecidos

El dataset de fallecidos será procesado con la librería **Pandas**.

### 2.1. Leer el dataset

Como se sabe que cada fila representa a una persona, solamente se toma la columna **'fecha de fallecimiento'**.

In [61]:
# Leer solamente la columna fecha de fallecimiento
fal_col = ['FECHA_FALLECIMIENTO']
df_fal = pd.read_csv(fal_url, sep = ';', usecols = fal_col, dtype = {'FECHA_FALLECIMIENTO':'int32'})
del fal_col

# Transformamos a formato fecha
df_fal.loc[:,'FECHA_FALLECIMIENTO'] = pd.to_datetime(df_fal['FECHA_FALLECIMIENTO'], format = '%Y%m%d')

df_fal.head()

,FECHA_FALLECIMIENTO
0,2021-06-11
1,2021-03-17
2,2021-06-02
3,2021-07-03
4,2021-05-06


### 2.2. Procesar el dataset

Se añaden 3 columnas. El año, la semana epidemiológica correspondiente y un contador para contar cada caso de fallecimiento por COVID-19. Se elimina la columna **'fecha_fallecimiento'**.

In [62]:
# Agregamos el año y semana epidemilógica de cada fallecido
fn.date_to_epiweek(df_fal,'FECHA_FALLECIMIENTO') 

# Contador de casos (cada 1 es un fallecido)    
df_fal['fallecidos'] = 1                                   
df_fal = df_fal.astype({'year': 'int16', 'epi_week': 'int8','fallecidos': 'int8'})

del df_fal['FECHA_FALLECIMIENTO']
df_fal.head()

,year,epi_week,fallecidos
0,2021,23,1
1,2021,11,1
2,2021,22,1
3,2021,26,1
4,2021,18,1


In [63]:
# Data type
df_fal.dtypes

year          int16
epi_week       int8
fallecidos     int8
dtype: object

Se cuenta cada caso de la variable **'fallecidos'** respecto al año y semana epidemiológica. 

In [64]:
# Devuelve un dataframe con el total de FALLECIDOS por año y semana epidemiológica
epi_fal = df_fal.groupby(['year', 'epi_week']).count()
del df_fal

epi_fal.head()

fallecidos
year epi_week            
2020 10                 3
     11                 3
     12                36
     13                64
     14               226

Se obtiene un dataframe **'epi_fal'** con el total de fallecidos por año y semana epidemiológica de todo el Perú.

## 3. Procesar el dataset de vacunados

### 3.1. Leer el dataset

Debido al tamaño del dataset de vacunados y para preservar recursos, no se procesará la información usando **Pandas**. En su lugar se utilizará el paquete **Dask**. La variable no será un **pandas dataframe** si no un **dask dataframe** el cual está separado en n particiones de tipo **pandas dataframe**.

In [72]:
# Leemos solo la columna fecha vacunación y dosis del dataset de vacunados usando dask
vac_col = ['fecha_vacunacion','dosis']
df_vac = dd.read_csv(vac_url, sep = ",", usecols = vac_col, 
                     dtype = {'fecha_vacunacion':'int32', 'dosis': 'int8'})

# Déjamos solamente a personas con 3 o menos dosis
df_vac = df_vac[df_vac['dosis'] <= 3]

# Convertimos a formato fecha
df_vac = df_vac.assign(fecha_vacunacion = dd.to_datetime(df_vac["fecha_vacunacion"], format = "%Y%m%d", 
                                                         errors="coerce"))

del vac_col
df_vac.head()

,fecha_vacunacion,dosis
0,2021-07-01,2
1,2021-07-03,2
2,2021-11-10,1
3,2021-11-26,2
4,2021-09-08,1


In [75]:
# Data types
df_vac.dtypes

fecha_vacunacion    datetime64[ns]
dosis                         int8
dtype: object

### 3.2. Procesar el dataset

Se añaden 2 columnas. El año y la semana epidemiológica correspondiente para cada vacunado contra COVID-19. Se elimina la columna **'fecha_fallecimiento'**.

In [67]:
# Obtenemos el año y semana epidemiológica en una sola columna
df_vac['epi_date'] = df_vac['fecha_vacunacion'].map(lambda date : Week.fromdate(date).isoformat())
del df_vac['fecha_vacunacion']

# Se separa la columna obtenida en 2, una para el año y otra para la semana epidemiológica
df_vac[['year','epi_week']] = df_vac['epi_date'].str.split("W", 1, expand=True)
del df_vac['epi_date']

# Cambiamos el tipo de variable para consumir menos recursos
df_vac['year'] = df_vac['year'].astype('int16')
df_vac['epi_week'] = df_vac['epi_week'].astype('int8')

df_vac.head()

,dosis,year,epi_week
0,2,2021,26
1,2,2021,26
2,1,2021,45
3,2,2021,47
4,1,2021,36


### 3.3. Crosstab del número de dosis (1,2 y 3) respecto al año y semana epidemiológica

**Nota:** No se puede usar multi-index en pivot_table para dask, debido a eso se crea la función **'crosstab4dask'**. Aún es necesario buscar en dask funciones para iterar todas las particiones más facilmente y optimizar el script **(Pendiente)**.

In [68]:
def crosstab4dask(ddf):
    """Función que recibe un dask dataframe (ddf) y realiza conteos de variables en cada partición
    y devuelve el total por los index que hayamos ingresado, en este caso 'year' y 'epi_week'."""
    
    lst = [] # Lista para almacenar la sumatoria de cada particion

    for i in range(0, ddf.npartitions):
        ddf = df_vac.partitions[i].compute()
        lst.append(pd.crosstab(index=[ddf['year'],ddf['epi_week']], columns = ddf['dosis']))

    merged_epivac = pd.concat(lst, axis=1)  # Merge all dfs
    del lst

    merged_epivac = merged_epivac.fillna(0).astype(np.int64)
    merged_epivac = merged_epivac.groupby(level=0, axis=1).sum()
    
    return merged_epivac

In [69]:
vac_dose = crosstab4dask(df_vac)
vac_dose

dosis                1        2       3
year epi_week                          
2021 6          105769        8       0
     7           92319        2       0
     8          123945       19       0
     9           35244    93107       0
     10          74408    81267       0
     11         133824   121219       0
     12          80380    39044       0
     13          25475    65857       0
     14          69645   103005       0
     15          56647   111846       0
     16         118153    29980       0
     17         278921    61044       0
     18         284016    51675       0
     19         395133   109529       0
     20         453124   174980       0
     21         564758   240987       0
     22         318197   210492       0
     23         516334   562163       0
     24         253140   621385       0
     25         344819   366890       0
     26         545263   449713       0
     27        1039249   259262       0
     28         963734   322684       0
     29         881714   485371       0
     30         426940   831709       0
     31         675581   932754       0
     32         489345   688637       0
     33         437032   597369       0
     34         633455   585400       0
     35         890810   556677       0
     36        1939110   397897       0
     37        1843101   541208       0
     38         849086   730745       0
     39         791200  1454091       0
     40         593589  1576007       0
     41         745099   905703   45572
     42         592790   728863  128174
     43         489069   560887   61568
     44        1148584   398542   57107
     45         840139   609340   84988
     46         720438   602235   74069
     47         562319   971340  252823
     48         404976   790176  667473
     49         163690   270282  213022

Se obtiene un dataset **'vac_dose'** con el número de dosis aplicada por año y semana epidemiológica de todo el Perú.

## 4. Unir los dataframes de fallecidos y dosis de vacuna por año y semana epidemiológica

In [70]:
# Unimos ambos dataframes por semana epidemiológica
epiweeks = pd.concat([epi_fal, vac_dose], axis=1)  

epiweeks = epiweeks.fillna(value = 0) # Rellenamos vacíos o Nan values con 0
del epi_fal, vac_dose

# Cambiamos a int ya que existen Nan values y cambia a float64 automáticamente
epiweeks = epiweeks.astype('int64')

# Rename columns
epiweeks.rename(columns={1: 'dosis_1', 2: 'dosis_2', 3: 'dosis_3'}, inplace=True)
                                    
epiweeks

fallecidos  dosis_1  dosis_2  dosis_3
year epi_week                                       
2020 10                 3        0        0        0
     11                 3        0        0        0
     12                36        0        0        0
     13                64        0        0        0
     14               226        0        0        0
...                   ...      ...      ...      ...
2021 45               210   840139   609340    84988
     46               241   720438   602235    74069
     47               249   562319   971340   252823
     48               257   404976   790176   667473
     49               121   163690   270282   213022

[93 rows x 4 columns]

## 6. Guardar el dataframe

In [71]:
epiweeks.to_csv('Data/TOTAL_vacunados_y_fallecidos_x_semanaEpi.csv')
del epiweeks

Finalmente se guarda el dataset final con el total de vacunados y fallecidos por año y semana epidemiológica de todo el Perú.